# Sincronización de procesos


## Semáforos

In [9]:
//%cflags:-lpthread -lm

/** 	compilar con math.h   -lm
** 	compilar con threads: -lpthread
*/

#include <stdio.h>
#include <stdlib.h>
#include <fcntl.h>
#include <time.h>
#include <math.h>
#include <pthread.h>
#include <semaphore.h>
#include <unistd.h>

#define MAX_PROCESOS 1
/* Variables para semaforos*/
sem_t *semaforo;

struct datos_tipo{
	int dato;
	int p;
}


* proceso(void *datos)
{
	struct datos_tipo *datos_proceso;
	datos_proceso = (struct datos_tipo *) datos;
	int a, i, j, p;
    sem_wait(semaforo);
	a = datos_proceso -> dato;
	p = datos_proceso -> p;
	for(i=0; i <= p; i++)
	{printf("%i ",a);}

	fflush(stdout);
	sleep(2);
	for(i=0; i<=p; i++)
	{
		printf("- ");
	}
	fflush(stdout);
    sem_post(semaforo);
}

int main()
{
	int error;
	char *valor_devuelto;
	/* Variables para hilos*/
	struct datos_tipo hilo1_datos, hilo2_datos;
	pthread_t idhilo1, idhilo2, idhilo3, idhilo4;

	hilo1_datos.dato = 1;
	hilo2_datos.dato = 2;
	hilo1_datos.p = 10;
	hilo2_datos.p = 5;

	semaforo = sem_open("semaforo_name", O_CREAT, 0700, MAX_PROCESOS);

	error=pthread_create(&idhilo1, NULL, (void *)proceso, (void *)(&hilo1_datos));
	if (error != 0)
	{
		perror ("No puedo crear hilo");
		exit (-1);
	}

	error=pthread_create(&idhilo2, NULL, (void *)proceso, (void *)(&hilo2_datos));
	if (error != 0)
	{
		perror ("No puedo crear thread");
		exit (-1);
	}
	pthread_join(idhilo2, (void **)&valor_devuelto);
	pthread_join(idhilo1, (void **)&valor_devuelto);

	sem_close(semaforo);
	sem_unlink("semaforo_name");
	return 0;
}


1 1 1 1 1 1 1 1 1 1 1 - - - - - - - - - - - 2 2 2 2 2 2 - - - - - - 

## Mutex

In [12]:
//%cflags:-lpthread -lm

/* Ejemplo de manejo de hilos
* 	compilar con threads: -lpthread
*/
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <unistd.h>


#define MAX_PROCESOS 1
#define NUM_HILOS    8

struct datos_tipo
{
	int dato;
	int p;
};

pthread_mutex_t mutex;


void * proceso(void *datos)
{
	struct datos_tipo *datos_proceso;
	datos_proceso = (struct datos_tipo *) datos;
	int a, i, j, p;
	
	a = datos_proceso -> dato;
	p = datos_proceso -> p;
    pthread_mutex_lock(&mutex);
	for(i=0; i <= p; i++)
	{
		printf("%i ",a);
	}
	fflush(stdout);
	sleep(3);
	for(i=0; i<=p; i++)
	{
		printf("- ");
	}
	fflush(stdout);
    pthread_mutex_unlock(&mutex);
}

int main()
{
int error, i;
char *valor_devuelto;

    /* Variables para hilos*/
    struct datos_tipo hilo_datos[NUM_HILOS];
    pthread_t idhilo[NUM_HILOS];

    for(i=0; i<NUM_HILOS; i++){
        hilo_datos[i].dato = i;
        hilo_datos[i].p = i+1;
    }
pthread_mutex_init(&mutex, NULL);

	/*se lanzan los hilos*/
    for(i=0; i<NUM_HILOS; i++){
        error=pthread_create(&idhilo[i], NULL, (void *)proceso, (void *)(&hilo_datos[i]));
        if (error != 0)
        {
	        perror ("No puedo crear hilo");
	        exit (-1);
        }
    }
    /*Esperar a que terminen*/
    for(i=0; i<NUM_HILOS; i++){
        pthread_join(idhilo[i], (void **)&valor_devuelto);
    }
pthread_mutex_destroy(&mutex);

return 0;
}

0 0 - - 1 1 1 - - - 2 2 2 2 - - - - 4 4 4 4 4 4 - - - - - - 3 3 3 3 3 - - - - - 5 5 5 5 5 5 5 - - - - - - - 6 6 6 6 6 6 6 6 - - - - - - - - 7 7 7 7 7 7 7 7 7 - - - - - - - - - 

In [13]:
//%cflags:-lpthread -lm

/** ejemplo de mutex en problema consumidor productor
** 	compilar con threads: gcc mutex-test.c -o mutex-test -lpthread
*/

#include <stdio.h>
#include <termios.h>
#include <sys/mman.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <time.h>
#include <math.h>
#include <pthread.h>
#include <unistd.h>

#define MAX_BUFFER 16
#define MAX_DATOS 32

pthread_mutex_t mutex;
pthread_cond_t vacio, lleno;
int buffer[MAX_BUFFER], num_datos;

// proceso consumidor
void * consumidor()
{
	struct datos_tipo *datos_proceso;
	int a, i, j, p, posicion=0, dato;

	for(i=0; i < MAX_DATOS; i++)
	{
	 pthread_mutex_lock(&mutex);
	 while(num_datos == 0)
	 	pthread_cond_wait(&vacio, &mutex);

	 dato = buffer[posicion];
	 if(posicion == 15) 
	 	posicion = 0;
	 else
	 	posicion ++;
	 
	 num_datos --;
	 
	 if(num_datos == MAX_BUFFER -1)
	 	pthread_cond_signal(&lleno);

	 pthread_mutex_unlock(&mutex);
	 printf("\nse ha consumido el dato: %d", dato);
	 fflush(stdout);	 	
	 sleep(1);	 
	}
	pthread_exit(0);


//
}

//proceso productor
void * productor()
{
	struct datos_tipo *datos_proceso;
	int a, i, j, p, posicion = 0, dato;

	for(i=0; i<MAX_DATOS; i++)
	{
	 	pthread_mutex_lock(&mutex);
	 	while(num_datos == MAX_BUFFER)
	 		pthread_cond_wait(&lleno, &mutex);
	
		buffer[posicion] = i;
		dato = i;

		if(posicion == 15) 
	 		posicion = 0;
	 	else
	 		posicion ++;
		num_datos ++;
		if(num_datos == 1)	
			pthread_cond_signal(&vacio);

		pthread_mutex_unlock(&mutex);	 
		printf("\nse ha producido el dato: %d", dato);
	 	fflush(stdout);
//	 	sleep(1);
	}
	pthread_exit(0);
//
}


int main()
{
int error;
char *valor_devuelto;

/* Variables para hilos*/
pthread_t idhilo1, idhilo2, idhilo3, idhilo4;

pthread_mutex_init(&mutex, NULL);
pthread_cond_init(&lleno, NULL);
pthread_cond_init(&vacio, NULL);

error=pthread_create(&idhilo1, NULL, (void *)productor, NULL);
if (error != 0)
{
	perror ("No puedo crear hilo");
	exit (-1);
}

error=pthread_create(&idhilo2, NULL, (void *)consumidor, NULL);
if (error != 0)
{
	perror ("No puedo crear thread");
	exit (-1);
}

pthread_join(idhilo2, (void **)&valor_devuelto);
pthread_join(idhilo1, (void **)&valor_devuelto);
pthread_mutex_destroy(&mutex);
pthread_cond_destroy(&lleno);
pthread_cond_destroy(&vacio);
return 0;
}



se ha producido el dato: 0
se ha producido el dato: 1
se ha producido el dato: 2
se ha producido el dato: 3
se ha producido el dato: 4
se ha producido el dato: 5
se ha producido el dato: 6
se ha producido el dato: 7
se ha producido el dato: 8
se ha producido el dato: 9
se ha producido el dato: 10
se ha producido el dato: 11
se ha producido el dato: 12
se ha producido el dato: 13
se ha producido el dato: 14
se ha producido el dato: 15
se ha consumido el dato: 0
se ha producido el dato: 16
se ha consumido el dato: 1
se ha producido el dato: 17
se ha consumido el dato: 2
se ha producido el dato: 18
se ha consumido el dato: 3
se ha producido el dato: 19
se ha consumido el dato: 4
se ha producido el dato: 20
se ha consumido el dato: 5
se ha producido el dato: 21
se ha consumido el dato: 6
se ha producido el dato: 22
se ha consumido el dato: 7
se ha producido el dato: 23
se ha consumido el dato: 8
se ha producido el dato: 24
se ha consumido el dato: 9
se ha producido el dato: 25
se ha consu